python:3.9.13

tensorflow:2.18.0

----

每局只取1个样本：慢  

缓慢降温增加探索性：效果不好。奖励只在最后一步才出现，探索性破坏奖励信号？  

更小卷积核：测试效果很差  

q-learning：测试效果很差  

sarsa：测试效果很差  

补充人工信息：效果提升

增加层数：加了两层，大部分时候表现优于aug，但是有交叉。

降低 learning rate: 从 1e-3 降低到 1e-4, 表现提升。

loss 修正: loss_factor = loss_factor = (len(xs_batch) / batch_size) ** 0.5  似无优势，可能训练过程略微平稳一些

----

目前aug(base4基础上，输入数据增加了人工信息) 略优于 base4(补上了base2遗漏的batch normalization 和 dropout)

bigger(增加了两层) 略优于 aug

出牌信息里去掉带牌？可能意义不大

为什么训练过程中表现有起伏呢？降低 learning rate？增加每个轮次的局数？DQN？

如果DQN的话，要怎么做呢？DQN使用 experience buffer 是为了降低样本的相关性。在斗地主中，如果我一次收集很多局的样本，然后把样本打乱，喂给模型，这样就达到了降低样本相关性的目的吧。在这种情况下，policy network 本身就不是每一步都更新的。还有必要区分 policy network 和 target network 吗？

按照DQN的逻辑，我是不是可以把同一批样本反复随机分割成batch喂给模型？可以加快训练速度吗？

之前 q-learning 的效果太差了，完全没信心

----

trainer输出的loss意义有限。如果发牌高度偏向于地主或农民，那么预测就更准，loss就会更小。一轮只有8局，发牌的影响很大。

----

感觉 DouZero 所谓 Deep Monte-Carlo 跟我的逻辑是一样的。打牌，生成一整局游戏的样本，用最终的输赢作为预测目标，训练状态动作价值模型。多线程不熟悉代码不怎么看得懂，似乎一次生成50局游戏，然后 batch size 32

----
因为电脑是8核，之前都是每8局游戏更新模型。地主农民分三个模型之后，表现下降。可能是因为每个模型的样本数量只有之前的1/3。相应地改成每24局游戏更新一次，似乎表现提升，但是训练速度当然慢了很多。

一个问题，把收集到的样本打乱，32个样本一个batch，不满32个的也当成一个batch，是不是有问题？不满32个的应该怎么处理。假如某组只有1个样本，是不是不太好？

----

怎么把douzero模型下载并跑起来……

----

tensorflow 模型 1696257 参数，运行速度 13ms，torch 模型 1457665 参数，运行速度 4ms。另外的我的模型需要的训练数据，组装速度也较慢，主要是组装额外数据(aug加入)消耗了额外的时间。

----
torch 比 tensorflow 快很多。发现用 torch 时，如果 batch size 为 1 会报错。所以 bn 是在一个 batch 内做 normalization？感觉不对劲。  
BatchNorm2D: batch 内所有样本同一个 channel 所有数据做 normalization, scaled(x[:, idx_channel, :, :])  
LayerNorm: batch 内每个样本单独做 normalization, scaled(x[idx_sample, :, :, :])  
算了，改用 LayerNorm 吧  
<!-- 发现 LayerNorm 更平稳。为什么？~~直觉上似乎 batch normalization 更合理？因为有少数 batch size 不是固定的32？ batch size 太小？~~ _之前 eval 局数更少_ -->
----

似乎遇到内存泄露，跑了一晚上，报错停止了。  
使用 with mp.Pool() as pool, 内存稳定，但速度慢很多  
使用 del, 无效  
使用 del 和 gc.collect(), 无效，速度慢  
将就一下，仍旧使用 pool = mp.Poo() 以及每200轮关闭和重建创建 pool

----

公司个人办公电脑效率测试：trainer_torch_ln, 两分钟
cpu| rounds
----|----
1| 59
2| 79
3| 90
4| 94
5| 98
6| 102

公司linux服务器效率测试：trainer_torch_ln, 两分钟
cpu|rounds
----|----
1| 50
2| 58
3| 38
4| 13
6| 4

同样的代码，在我的公司电脑上运行的速度是在公司服务器/阿里云服务器上的20多倍。为什么？linux的问题？从资源占用来看，服务器上的cpu也是一直在工作的。  
找到原因了。模型计算时默认使用多核，各进程的模型互相抢CPU。不知道为什么这个问题在个人PC上没那么严重，windows和linux的差异？  
torch.set_num_threads 设置使用模型使用的线程数。在linux服务器上设置12进程，子进程 torch.set_num_threads(1) 之后，两分钟跑了80轮  
本地PC采用8进程和torch.set_num_threads(1)之后，运行速度似有有些微提升。

-----
使用 python multiprocessing 多进程遇到的问题：  
内存占用不断增加：使用 with mp.Pool() as pool 可以避免这个问题，但是会降低程序运行速度。改成在循环中每隔若干轮关闭并重建pool。  
使用 Process(target, args)，不一定使用多核。不知道如何指定使用几个进程。即使使用多核，使用队列传递数据，程序运行速度相比 pool.map 也不见优势。  
使用多进程，运行速度没有线性增加。  
使用6核，设定每个 selfplay 进行一局游戏，执行24次，或者设定每个selfplay 进行4局游戏，执行6次，运行效率没区别。但把总执行次数改成48局，前一种会稍慢。说明复制参数或收集返回结果确实有计算成本，只是在本电脑本问题上不是很显著。  

----
Dropout: 同样的输入，在 train 模式下，输出会不一样。但是如果输出的方差很大，这会不会是个问题？  
刚才是用随机生成的数据测试。重新用实际的牌局数据测试，方差不大。





In [15]:
import numpy as np
import keras as k
import tensorflow as tf
import torch
from torch import nn
import rules
import arena as arn
import bot_torch_ln
import bot_aug
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [1]:
import bot_douzero
import importlib
importlib.reload(bot_douzero)

<module 'bot_douzero' from 'd:\\py\\ddz_2025_2\\bot_douzero.py'>

In [3]:
def ft(N, bot1_f, bot2_f):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s1 += arena.winner == 0

        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena.wholegame()
        s2 += arena.winner == 0

    return s1, s2

# models & bots

In [ ]:
import randbot_three

<module 'bot_three' from 'd:\\py\\ddz_2025_2\\bot_three.py'>

In [98]:
cp = torch.load("model_torch_ln/cp144000.pt")
models = []
for pos in range(3):
    model = bot_torch_ln.Model()
    model.eval()
    model.load_state_dict(cp["models_state_dict"][pos])
    models.append(model)

In [99]:
def bot_torch_ln_f(verbos=0):
    return bot_torch_ln.BOT(models, verbos=verbos)

def bot_rand_f(verbos=0, sample=10):
    return randbot_three.BOT([bot_torch_ln_f(0), bot_torch_ln_f(0), bot_torch_ln_f(0)], verbos=verbos, sample=sample)

model_aug = k.models.load_model("e:/ddz_2025_2_model/model_aug/m488750.keras")
def bot_aug_f(verbos=0):
    return bot_aug.BOT(model_aug, verbos)

def bot_rd_f():
    return bot_rd

def bot_douzero_f(verbos=0):
    return bot_douzero.BOT(verbos)

In [100]:
N = 100
# ft(N, bot_three_f, bot_three_f)  ##623
# ft(N, bot_douzero_f, bot_douzero_f) ##66
ft(N, bot_torch_ln_f, bot_douzero_f)

(26, 25)

In [101]:
arena = arn.ARENA(1, True)
arena.registerbot([bot_torch_ln_f(1), bot_douzero_f(1), bot_douzero_f(1)])
arena.wholegame()

0.015563988 4
0.014944937 3 4 5 6 7 8 9 10 J
0.014271956 7
0.013472806 7 8 9 10 J
0.013453123 J
 0 :  地主 : 4 | 3 4 5 6 6 6 7 7 8 9 10 10 J J Q K A A A
0.915523 8
0.88073564 10
0.8744279 2
0.87231827 9
0.85951054 Q
 1 : 农民1 : 8 | 3 4 4 5 6 7 8 9 10 Q Q K K 2 小王 大王
0.3483035 A
0.32748893 K
0.2914596 Q
0.27462292 
0.2702033 10
 2 : 农民2 : A | 3 3 5 5 7 8 9 9 10 J J Q K 2 2 2
0.016451363 
 3 :  地主 :  | 3 4 5 6 6 6 7 7 8 9 10 10 J J Q K A A A
0.9432374 2
0.91718674 
0.6217257 小王
0.4746996 大王
0.3291227 小王 大王
 4 : 农民1 : 2 | 3 4 4 5 6 7 8 9 10 Q Q K K 小王 大王
 5 : 农民2 :  | 3 3 5 5 7 8 9 9 10 J J Q K 2 2 2
0.019564431 
 6 :  地主 :  | 3 4 5 6 6 6 7 7 8 9 10 10 J J Q K A A A
0.9461405 3 4 5 6 7 8 9 10
0.8988652 Q Q
0.8867767 K K
0.8608748 3 4 5 6 7 8 9
0.84013087 4 5 6 7 8 9 10
 7 : 农民1 : 3 4 5 6 7 8 9 10 | 4 Q Q K K 小王 大王
 8 : 农民2 :  | 3 3 5 5 7 8 9 9 10 J J Q K 2 2 2
0.011575426 
0.011119186 4 5 6 7 8 9 10 J
0.010964742 6 7 8 9 10 J Q K
0.009713727 7 8 9 10 J Q K A
0.005768888 5 6 7 8 9 10 J Q
 9 :

In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

# 草稿